In [59]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd

Fixed Model Parameters

In [60]:
alpha_1 = 1e-3
alpha_2 = 1.44
fk_max = 1
R_k = 0.5
v_1 = 1.2e2
v_2 = 14.4 

Kill Function and Dimensionless ODE Model

In [61]:
def kill_function(R):
     f = fk_max*(R/(R + R_k))

     return f

def model(t, z, gamma, mu):

    dAdt = -alpha_1*(1 - z[1])*z[0]*z[2] + alpha_2*gamma*z[1]*z[2]
    dRdt = (alpha_1/gamma)*(1 - z[1])*z[0] - alpha_2*z[1]
    dSdt = -kill_function(z[1])*z[3]
    dCdt = v_1*(mu - z[3])*(z[2] - z[3]) - v_2*z[3] - kill_function(z[1])*z[3]
    dzdt = [dAdt, dRdt, dSdt, dCdt]

    return dzdt

ICs and Simulation Parameters

In [62]:
z0 = [1, 0, 1, 0] 
t = np.arange(0, 1.1, 0.1)
t_span = [0, 1]
mu_vals = [1.25, 2.5, 5, 10, 20]
gamma_vals = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
noise_vals = [0.01, 0.05, 0.1, 0.2]

Create Dataframe

In [63]:
df = pd.DataFrame(columns=['t', 'mu', 'gamma', 'noise level', 'A', 'R', 'S', 'C'])


In [64]:

for gamma in gamma_vals:
    for mu in mu_vals:
        for noise_level in noise_vals:
            z = solve_ivp(model, t_span, z0, args = (gamma, mu), method ='Radau', t_eval = t)

            # add noise
            A_array = z.y[0]
            R_array = z.y[1]
            S_array = z.y[2]
            C_array = z.y[3]

            for i in range(len(A_array)):
                A_array[i] = A_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                R_array[i] = R_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                S_array[i] = S_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                C_array[i] = C_array[i]*(1 + np.random.uniform(-noise_level, noise_level))

            # add data to dataframe
            for i in range(len(A_array)):
                df = df.append({'t' : t[i],
                                'mu' : mu,
                                'gamma' : gamma,
                                'noise level' : noise_level,
                                'A' : A_array[i],
                                'R' : R_array[i],
                                'S' : S_array[i],
                                'C' : C_array[i]}, ignore_index=True)

        

In [65]:
print(df)

        t     mu         gamma  noise level         A         R         S  \
0     0.0   1.25  1.000000e+00         0.01  0.998606  0.000000  1.000346   
1     0.1   1.25  1.000000e+00         0.01  1.007527  0.000093  0.998793   
2     0.2   1.25  1.000000e+00         0.01  1.002289  0.000174  1.009234   
3     0.3   1.25  1.000000e+00         0.01  0.990457  0.000242  0.995423   
4     0.4   1.25  1.000000e+00         0.01  0.990199  0.000307  1.000282   
...   ...    ...           ...          ...       ...       ...       ...   
1975  0.6  20.00  1.000000e-08         0.20  0.832006  0.854553  0.569774   
1976  0.7  20.00  1.000000e-08         0.20  0.867716  1.165525  0.514345   
1977  0.8  20.00  1.000000e-08         0.20  0.938947  0.967067  0.646961   
1978  0.9  20.00  1.000000e-08         0.20  0.867210  1.142501  0.495335   
1979  1.0  20.00  1.000000e-08         0.20  1.053632  0.964812  0.415522   

             C  
0     0.000000  
1     0.791728  
2     0.790517  
3     0

Write Dataframe to CSV

In [66]:
df.to_csv('basic_model_synthetic_data.csv')